In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Constants
G = 6.67430e-11  # Gravitational constant in N*m^2/kg^2
M_sun = 1.989e30  # Solar mass in kg
R_sun = 6.955e8  # Solar radius in m
#K_NR = 1.2435e15  # Constant for non-relativistic electron degenerate gas (cgs units)
m_e = 9.11e-31 #Electron mass in kg
hbar = 1.05457182e-34 #Reduced Planck constant in Js^-1
c = 2.99e8 #Speed of light in ms^-1
a_z = 2
m_n = 1.67e-27 #Neutron mass in kg

In [30]:
def K_R():
    """Inverse EOS to get density from pressure for relativistic case"""
    return (hbar* c)/ (12 * np.pi**2)  * (3 * np.pi**2 / (a_z * m_n * c**2))**(4/3)

def eos(P=None, rho=None,):
    # Check that only one of P or rho is provided
    if P is None and rho is None:
        raise ValueError("Either pressure (P) or density (rho) must be provided.")
    
    n = 4/3
    K = K_R()
    
    if P is not None:
        return (P / K) ** (1/n)  / c**2 # Density from pressure
    else:
        if rho is None:
            raise ValueError("Density rho must be provided for pressure calculation.")
        return K * (rho * c**2) ** n  # Pressure from density



# ----------------- Event to Stop at Pressure < threshold -----------------
def stop_at_pressure_threshold(r, y):
    threshold_pressure = 1e-3  # Define a small threshold for pressure
    return y[0] - threshold_pressure  # Stop when pressure y[0] < threshold_pressure
stop_at_pressure_threshold.terminal = True  # Stop integration when this condition is met
stop_at_pressure_threshold.direction = -1   # Trigger only when pressure is decreasing

# ----------------- Main Integration Function -----------------
def integrate_star(central_pressure):
    """Solve the structure equations numerically"""
    radius = 1e11

    densities = []
    print(len(densities))

    # ----------------- Structure Solver -----------------
    def hydrostatic_equilibrium(r, y):
        """Equation for hydrostatic equilibrium: y[0]=Pressure, y[1]=Mass"""
        P, M= y
        rho = eos(P=P)
        print(rho)
        densities.append(rho)

        dP_dr = (-G * M * rho) / (r**2)
        dM_dr = 4 * np.pi * r**2 * rho
        return [dP_dr, dM_dr]
    
    sol = solve_ivp(
        hydrostatic_equilibrium, 
        [1e-6, radius], 
        [central_pressure, 0],
        method='RK45',
        events=stop_at_pressure_threshold, 
        dense_output=True, 
        max_step=1e3, 
        rtol=1e-6, 
        atol=1e-8)
    
    
    
    num_bins = len(sol.t)

    
    return sol.t, sol.y[0], sol.y[1], densities, num_bins  # Return radius, pressure, and mass profiles